In [1]:
import numpy as np
import pandas as pd

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

In [4]:
concrete_data.describe

<bound method NDFrame.describe of       Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0      540.0                 0.0      0.0  162.0               2.5   
1      540.0                 0.0      0.0  162.0               2.5   
2      332.5               142.5      0.0  228.0               0.0   
3      332.5               142.5      0.0  228.0               0.0   
4      198.6               132.4      0.0  192.0               0.0   
...      ...                 ...      ...    ...               ...   
1025   276.4               116.0     90.3  179.6               8.9   
1026   322.2                 0.0    115.6  196.0              10.4   
1027   148.5               139.4    108.6  192.7               6.1   
1028   159.1               186.7      0.0  175.6              11.3   
1029   260.9               100.5     78.3  200.6               8.6   

      Coarse Aggregate  Fine Aggregate  Age  Strength  
0               1040.0           676.0   28     79.99  
1            

In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']]
target = concrete_data['Strength']

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
n_cols = predictors_norm.shape[1]

In [11]:
n_cols

8

In [12]:
import keras

Using TensorFlow backend.


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [14]:
def regression_model():
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [15]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size = 0.3)
model = regression_model()

In [17]:
model.fit(X_train, y_train, epochs=50, verbose=0)

y_pred = model.predict(X_test)

loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
print("loss of value : " + str(loss_val))
    
print("mean mse : " + str(np.mean(mean_squared_error(y_test, y_pred))))
print("std mse : " + str(np.std(mean_squared_error(y_test, y_pred))))
print("mean r2 : "+ str(np.mean(r2_score(y_test, y_pred))))
print("std r2 : "+ str(np.std(r2_score(y_test, y_pred))))

10/10 [==============================] - 0s 2ms/step - loss: 52.3653
loss of value : 52.36533737182617
mean mse : 52.3653381623213
std mse : 0.0
mean r2 : 0.8303394679945888
std r2 : 0.0


In [18]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 1238.8670654296875
MSE 2: 604.6317138671875
MSE 3: 257.9229431152344
MSE 4: 164.90740966796875
MSE 5: 107.56346130371094
MSE 6: 106.75945281982422
MSE 7: 103.17059326171875
MSE 8: 71.47161865234375
MSE 9: 77.9140396118164
MSE 10: 129.1336212158203
MSE 11: 78.2120590209961
MSE 12: 65.66136932373047
MSE 13: 94.88736724853516
MSE 14: 76.89706420898438
MSE 15: 71.38019561767578
MSE 16: 46.03037643432617
MSE 17: 52.64616775512695
MSE 18: 41.053810119628906
MSE 19: 82.08057403564453
MSE 20: 93.58013153076172
MSE 21: 40.096153259277344
MSE 22: 66.45557403564453
MSE 23: 82.72158813476562
MSE 24: 37.07855987548828
MSE 25: 34.078956604003906
MSE 26: 81.5718994140625
MSE 27: 59.99933624267578
MSE 28: 35.68748474121094
MSE 29: 44.25423049926758
MSE 30: 34.35519027709961
MSE 31: 99.2624282836914
MSE 32: 150.92063903808594
MSE 33: 35.07240676879883
MSE 34: 37.59549331665039
MSE 35: 35.27298355102539
MSE 36: 51.46199035644531
MSE 37: 48.06607437133789
MSE 38: 68.0398941040039
MSE 39: 40.478176